In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from client_info import SPOTIFY_CID as cid, SPOTIFY_CS as secret
import pandas as pd
import numpy as np
import track_methods

In [3]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


In [4]:
def get_all_tracks(user_id,playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [5]:
def get_playlist_uid(playlist_url):
    return playlist_url.split('/')[-1].split('?')[0]

get_playlist_uid('https://open.spotify.com/playlist/2YRe7HRKNRvXdJBp9nXFza?si=\198755290d2c4102'
)

'2YRe7HRKNRvXdJBp9nXFza'

In [6]:
#>=500 million plays 
#https://open.spotify.com/playlist/2YRe7HRKNRvXdJBp9nXFza?si=\198755290d2c4102
#user
#https://open.spotify.com/user/1221028518?si=2aa6f2f72ab44757

mil_playlist_url = 'https://open.spotify.com/playlist/2YRe7HRKNRvXdJBp9nXFza?si=\198755290d2c4102'
track_list = get_all_tracks('2aa6f2f72ab44757', mil_playlist_url)
#sp.playlist(mil_playlist_url)['tracks']['items']
len(track_list)

682

In [8]:
type(track_list)

list

In [11]:
#create list to create dataframe with. contains all features+song name and artist
track_info = []
bad_info = ['type', 'id', 'uri', 'track_href', 'analysis_url']
total_genres = set()
for track in track_list:
    track_uri=track['track']['uri']
    usable_track=sp.track(track_uri)
    track_id=usable_track['id']

    track_features = {}
    track_features['name'] = usable_track['name']
    track_features['artist(s)'] = [usable_track['artists'][i]['name'] 
        for i in range(len(usable_track['artists']))]

    artist_ids = [usable_track['artists'][i]['uri'] 
        for i in range(len(usable_track['artists']))]
    genres = [sp.artist(artist)['genres'] for artist in artist_ids]
    track_features['genres'] = genres
    [total_genres.update(genre) for genre in genres]

    track_info.append(track_features)


In [13]:
track_info

[{'name': 'Blinding Lights',
  'artist(s)': ['The Weeknd'],
  'genres': [['canadian contemporary r&b', 'canadian pop', 'pop']]},
 {'name': 'Shape of You',
  'artist(s)': ['Ed Sheeran'],
  'genres': [['pop', 'uk pop']]},
 {'name': 'Dance Monkey',
  'artist(s)': ['Tones And I'],
  'genres': [['australian pop', 'pop']]},
 {'name': 'Someone You Loved',
  'artist(s)': ['Lewis Capaldi'],
  'genres': [['pop', 'uk pop']]},
 {'name': 'rockstar (feat. 21 Savage)',
  'artist(s)': ['Post Malone', '21 Savage'],
  'genres': [['dfw rap', 'melodic rap', 'rap'], ['atl hip hop', 'rap']]},
 {'name': 'Sunflower - Spider-Man: Into the Spider-Verse',
  'artist(s)': ['Post Malone', 'Swae Lee'],
  'genres': [['dfw rap', 'melodic rap', 'rap'], ['melodic rap', 'trap']]},
 {'name': 'One Dance',
  'artist(s)': ['Drake', 'Wizkid', 'Kyla'],
  'genres': [['canadian hip hop',
    'canadian pop',
    'hip hop',
    'rap',
    'toronto rap'],
   ['afro dancehall', 'afropop', 'azonto', 'nigerian hip hop', 'nigerian pop'

In [14]:
reduced_genres = {}
for genre in total_genres:
    if ' ' not in genre or genre == "hip hop":
        reduced_genres[genre] = []

for genre in total_genres:
    base_genre = genre
    if ' ' in genre:
        for key in reduced_genres.keys():
            if key in genre:
                base_genre = key
    if base_genre not in reduced_genres:
        reduced_genres[base_genre] = []
    else:
        reduced_genres[base_genre].append(genre)

In [15]:
print(len(reduced_genres))
reduced_genres

110


{'moombahton': ['moombahton'],
 'electro': ['electro', 'canadian electronic'],
 'disco': ['disco'],
 'aussietronica': ['aussietronica'],
 'neo-psychedelic': ['neo-psychedelic'],
 'grunge': ['grunge'],
 'melancholia': ['melancholia'],
 'complextro': ['complextro'],
 'bubblegrunge': ['bubblegrunge'],
 'speedrun': ['speedrun'],
 'soul': ['soul', 'british soul', 'northern soul', 'neo soul', 'classic soul'],
 'edm': ['edm', 'dutch edm'],
 'pop': ['baroque pop',
  'danish pop',
  'pop dance',
  'bedroom pop',
  'modern alternative pop',
  'post-teen pop',
  'colombian pop',
  'hip pop',
  'pop soul',
  'dark pop',
  'dutch pop',
  'pop',
  'french pop',
  'norwegian pop',
  'australian pop',
  'indie pop',
  'puerto rican pop',
  'latin arena pop',
  'deep dance pop',
  'israeli pop',
  'latin viral pop',
  'pop edm',
  'uk alternative pop',
  'teen pop',
  'acoustic pop',
  'social media pop',
  'mexican pop',
  'pop electronico',
  'gauze pop',
  'dance pop',
  'uk pop',
  'shimmer pop',
 

In [16]:
tracks_df = pd.DataFrame.from_dict(track_info)
tracks_df

,name,artist(s),genres
0,Blinding Lights,[The Weeknd],"[[canadian contemporary r&b, canadian pop, pop]]"
1,Shape of You,[Ed Sheeran],"[[pop, uk pop]]"
2,Dance Monkey,[Tones And I],"[[australian pop, pop]]"
3,Someone You Loved,[Lewis Capaldi],"[[pop, uk pop]]"
4,rockstar (feat. 21 Savage),"[Post Malone, 21 Savage]","[[dfw rap, melodic rap, rap], [atl hip hop, rap]]"
...,...,...,...
677,Tiny Dancer,[Elton John],"[[glam rock, mellow gold, piano rock]]"
678,Gimme! Gimme! Gimme! (A Man After Midnight),[ABBA],"[[europop, swedish pop]]"
679,I'm Good (Blue),"[David Guetta, Bebe Rexha]","[[big room, dance pop, edm, pop, pop dance], [..."
680,California Dreamin' - Single Version,[The Mamas & The Papas],"[[bubblegum pop, classic rock, folk rock, mell..."
